In [13]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import math
import os
import random
import pandas as pd
import numpy as np
from sklearn import datasets, ensemble
from catboost import CatBoostRegressor
from tqdm import tqdm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate, train_test_split
import itertools
import math
import warnings
warnings.filterwarnings(action='ignore')

In [14]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,LENGTH,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE,CI_HOUR
0,TRAIN_000000,SG,GIW5,Container,30.881018,2018-12-17 21:29,Z618338,30.0,24,24300,...,180.0,CQSB78,Panama,NaN,NaN,NaN,NaN,5,0.002615,3.450000
1,TRAIN_000001,IN,UJM2,Bulk,0.000000,2014-09-23 6:59,X886125,30.0,13,35900,...,180.0,SPNO34,Marshall Islands,NaN,NaN,NaN,NaN,12,0.000217,0.000000
2,TRAIN_000002,CN,EUC8,Container,0.000000,2015-02-03 22:00,T674582,50.0,12,146000,...,370.0,FNPK22,Malta,NaN,NaN,NaN,NaN,6,0.001614,0.000000
3,TRAIN_000003,JP,ZAG4,Container,0.000000,2020-01-17 4:02,Y847238,20.0,18,6910,...,120.0,PBZV77,Bahamas,-3.18,-1.61,6.7,2.629350,13,0.000356,0.000000
4,TRAIN_000004,SG,GIW5,Container,27.037650,2020-01-26 7:51,A872328,50.0,10,116000,...,300.0,GUCE76,Liberia,-0.33,-3.28,25.6,2.495953,15,0.002615,253.554444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391934,TRAIN_391934,JP,QYY1,Container,0.000000,2017-06-06 5:02,Y375615,20.0,27,6820,...,110.0,KEJZ24,"China, People's Republic Of",NaN,NaN,NaN,NaN,14,0.000552,0.000000
391935,TRAIN_391935,SG,GIW5,Bulk,5.884603,2019-10-16 0:36,K635567,10.0,12,3160,...,80.0,JLTM64,Vietnam,-0.66,0.97,27.3,1.253491,8,0.002615,144.061389
391936,TRAIN_391936,US,QGN3,Bulk,70.660241,2021-03-23 22:35,J284147,30.0,8,60300,...,200.0,YERJ68,Singapore,-3.44,7.99,21.1,4.766257,18,0.000155,41.482222
391937,TRAIN_391937,TW,JWI3,Container,9.448179,2015-01-08 7:15,J644215,30.0,29,23800,...,170.0,HCZK58,Comoros,NaN,NaN,NaN,NaN,15,0.000990,7.485278


In [3]:
#train = train[train['DIST'] != 0]
#train = train.reset_index(drop = True)
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,DUBAI,BRENT,WTI,BDI_ADJ,PORT_SIZE,CI_HOUR
0,TRAIN_000000,CN,EKP8,Bulk,30.736578,2020-10-15 4:03,Z517571,30.0,28,73100,...,3.77,15.9,2.730798,12,42.01,43.16,40.96,1407.668330,0.001660,3.048333
1,TRAIN_000001,CN,EUC8,Container,63.220425,2019-09-17 2:55,U467618,30.0,15,37900,...,-6.72,24.5,4.289058,10,67.53,64.55,59.34,2089.046774,0.001614,17.138611
2,TRAIN_000002,CN,NGG6,Container,90.427421,2019-02-23 6:43,V378315,50.0,7,115000,...,0.00,9.4,0.000000,14,65.30,66.39,56.94,603.193047,0.001743,98.827500
3,TRAIN_000003,JP,TMR7,Cargo,0.000000,2020-09-18 22:06,B726632,10.0,33,1490,...,-7.31,22.1,4.693735,7,43.02,43.15,41.11,1169.853455,0.000069,0.000000
4,TRAIN_000004,RU,NNC2,Container,8.813725,2022-08-13 12:57,D215135,30.0,10,27600,...,2.31,22.8,2.345875,14,90.45,93.65,88.11,1107.944894,0.000197,96.030556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367436,TRAIN_367436,CN,YRT6,Bulk,59.018184,2017-11-11 22:23,J661243,40.0,13,93200,...,NaN,NaN,NaN,6,61.25,62.21,55.70,1333.609109,0.000360,65.850000
367437,TRAIN_367437,JP,QYY1,Tanker,0.000000,2022-04-29 2:58,D847216,10.0,9,1280,...,0.87,17.1,1.028558,11,105.37,109.34,104.69,1955.103846,0.000552,0.000000
367438,TRAIN_367438,SG,GIW5,Container,1.768630,2022-07-14 7:58,Q635545,30.0,6,25000,...,3.36,31.7,2.557156,15,97.73,99.10,95.78,1601.291086,0.002615,0.997500
367439,TRAIN_367439,JP,TMR7,Cargo,0.000000,2020-12-22 10:07,N211282,10.0,8,2400,...,-2.44,10.8,3.055715,19,49.75,50.08,47.02,1191.353331,0.000069,0.000000


In [15]:
train.drop(['SAMPLE_ID'],axis=1,inplace=True)
test.drop(['SAMPLE_ID'],axis=1,inplace=True)

In [16]:
train['ATA'] = pd.to_datetime(train['ATA'])
test['ATA'] = pd.to_datetime(test['ATA'])
train['year'] = train['ATA'].dt.year
train['month'] = train['ATA'].dt.month
train['day'] = train['ATA'].dt.day
train['weekday'] = train['ATA'].dt.dayofweek
train['rounded_hour'] = (train['ATA'].dt.hour + (train['ATA'].dt.minute // 30)).apply(lambda x: 0 if x == 24 else x)
test['year'] = test['ATA'].dt.year
test['month'] = test['ATA'].dt.month
test['day'] = test['ATA'].dt.day
test['weekday'] = test['ATA'].dt.dayofweek
test['rounded_hour'] = (test['ATA'].dt.hour + (test['ATA'].dt.minute // 30)).apply(lambda x: 0 if x == 24 else x)

# sin, cos 변환 함수 정의
def encode_cyclic_feature(data, column, max_val):
    data[column + '_sin'] = np.sin(2 * np.pi * data[column] / max_val)
    data[column + '_cos'] = np.cos(2 * np.pi * data[column] / max_val)
    return data

# 각 피처에 대해 sin, cos 변환 수행
train = encode_cyclic_feature(train, 'month', 12)
train = encode_cyclic_feature(train, 'day', 31)
train = encode_cyclic_feature(train, 'weekday', 7)
train = encode_cyclic_feature(train, 'rounded_hour', 24)
test = encode_cyclic_feature(test, 'month', 12)
test = encode_cyclic_feature(test, 'day', 31)
test = encode_cyclic_feature(test, 'weekday', 7)
test = encode_cyclic_feature(test, 'rounded_hour', 24)

train.drop(['ATA'],axis=1,inplace=True)
test.drop(['ATA'],axis=1,inplace=True)

test[['year', 'month', 'day', 'weekday', 'rounded_hour']].head()

,year,month,day,weekday,rounded_hour
0,2020,6,18,3,12
1,2021,5,26,2,22
2,2019,12,16,0,0
3,2015,11,16,0,6
4,2018,10,24,2,1


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244989 entries, 0 to 244988
Data columns (total 37 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ARI_CO              244989 non-null  object 
 1   ARI_PO              244989 non-null  object 
 2   SHIP_TYPE_CATEGORY  244989 non-null  object 
 3   DIST                244989 non-null  float64
 4   ID                  244989 non-null  object 
 5   BREADTH             244989 non-null  float64
 6   BUILT               244989 non-null  int64  
 7   DEADWEIGHT          244989 non-null  int64  
 8   DEPTH               244989 non-null  float64
 9   DRAUGHT             244989 non-null  float64
 10  GT                  244989 non-null  int64  
 11  LENGTH              244989 non-null  float64
 12  SHIPMANAGER         244989 non-null  object 
 13  FLAG                244989 non-null  object 
 14  U_WIND              143062 non-null  float64
 15  V_WIND              143062 non-nul

In [7]:
train = train[['WTI', 'DUBAI', 'BRENT', 'month_sin', 'day_sin', 'BDI_ADJ', 'CI_HOUR', 'ARI_CO', 'ARI_PO', 'ATA_LT', 'DIST']].copy()
test = test[['WTI', 'DUBAI', 'BRENT', 'month_sin', 'day_sin', 'BDI_ADJ' , 'ARI_CO', 'ARI_PO', 'ATA_LT', 'DIST']].copy()

In [8]:
train = train[['WTI', 'DUBAI', 'BRENT', 'month_sin', 'day_sin', 'BDI_ADJ', 'CI_HOUR']].copy()
test = test[['WTI', 'DUBAI', 'BRENT', 'month_sin', 'day_sin', 'BDI_ADJ']].copy()

In [9]:
train_x = train.drop(['CI_HOUR'],axis=1)
train_y = train[['CI_HOUR']]

# AG

## 5fold

In [53]:

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
'''
# ARI_CO + weekday 결합 열 생성
train['fold_column'] = train['ARI_CO'].astype(str) + "_" + train['weekday'].astype(str)

# Stratified 5-fold 생성
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=313)

# 각 폴드마다의 예측값과 MAE를 저장할 리스트
predictions = []
mae_scores = []

# StratifiedKFold는 레이블 분포를 기반으로 하기 때문에, 너무 적은 빈도의 조합은 오류를 발생시킬 수 있습니다.
min_samples = train['fold_column'].value_counts().min()
filtered_train = train[train['fold_column'].map(train['fold_column'].value_counts()) > min_samples]

for train_idx, val_idx in skf.split(filtered_train, filtered_train['fold_column']):
    train_fold = filtered_train.iloc[train_idx].drop(columns=['fold_column'])
    val_fold = filtered_train.iloc[val_idx].drop(columns=['fold_column'])
    train_data = TabularDataset(train_fold)
    val_data = TabularDataset(val_fold)
    test_data = TabularDataset(test)
    
    # 각 폴드마다 모델 학습
    predictor = TabularPredictor(label='CI_HOUR', eval_metric='mean_absolute_error').fit(
        train_data, 
        presets='medium_quality',
        ag_args_fit={'num_gpus': 0},
        included_model_types=['CAT']
    )
    
    # validation set에 대한 예측값 계산
    val_predictions = predictor.predict(val_data.drop(columns='CI_HOUR'))
    
    # MAE 계산 후 저장
    mae = mean_absolute_error(val_data['CI_HOUR'], val_predictions)
    mae_scores.append(mae)
    
    # test set에 대한 예측값 저장
    predictions.append(predictor.predict(test_data))

# 예측값의 평균 계산
final_predictions = np.mean(predictions, axis=0)
y_pred = pd.DataFrame(final_predictions, columns=['CI_HOUR'])
submission['CI_HOUR'] = y_pred['CI_HOUR']
mae_scores, np.mean(mae_scores)
'''

'\n# ARI_CO + weekday 결합 열 생성\ntrain[\'fold_column\'] = train[\'ARI_CO\'].astype(str) + "_" + train[\'weekday\'].astype(str)\n\n# Stratified 5-fold 생성\nskf = StratifiedKFold(n_splits=5, shuffle=True, random_state=313)\n\n# 각 폴드마다의 예측값과 MAE를 저장할 리스트\npredictions = []\nmae_scores = []\n\n# StratifiedKFold는 레이블 분포를 기반으로 하기 때문에, 너무 적은 빈도의 조합은 오류를 발생시킬 수 있습니다.\nmin_samples = train[\'fold_column\'].value_counts().min()\nfiltered_train = train[train[\'fold_column\'].map(train[\'fold_column\'].value_counts()) > min_samples]\n\nfor train_idx, val_idx in skf.split(filtered_train, filtered_train[\'fold_column\']):\n    train_fold = filtered_train.iloc[train_idx].drop(columns=[\'fold_column\'])\n    val_fold = filtered_train.iloc[val_idx].drop(columns=[\'fold_column\'])\n    train_data = TabularDataset(train_fold)\n    val_data = TabularDataset(val_fold)\n    test_data = TabularDataset(test)\n    \n    # 각 폴드마다 모델 학습\n    predictor = TabularPredictor(label=\'CI_HOUR\', eval_metric=\'mean_absolute_

In [56]:
from autogluon.tabular import TabularDataset, TabularPredictor

# Load train, test, and sample_submission data
train_data = train.copy()
test_data = test.copy()
sample_submission = submission.copy()

# Create an empty DataFrame for the final predictions
final_predictions = pd.DataFrame(index=test_data.index)
final_predictions['CI_HOUR'] = 0  # Initialize with 0

# Filter the ARI_PO values in train_data that have at least 400 occurrences
ari_po_values_to_model = train_data['ARI_PO'].value_counts()
ari_po_values_to_model = ari_po_values_to_model[ari_po_values_to_model >= 400].index.tolist()

# Loop through each ARI_PO value and train a model for it
for ari_po in ari_po_values_to_model:
    train_subset = train_data[train_data['ARI_PO'] == ari_po]
    test_subset = test_data[test_data['ARI_PO'] == ari_po]
    print(ari_po)
    print(ari_po)
    print(ari_po)
    print(ari_po)
    print(ari_po)

    print(len(train_subset))
    print(len(train_subset))
    print(len(train_subset))
    print(len(train_subset))
    print(len(train_subset))
    
    # Convert to AutoGluon Dataset
    train_subset_ag = TabularDataset(train_subset)
    test_subset_ag = TabularDataset(test_subset)

    # Train the model
    predictor = TabularPredictor(label='CI_HOUR', eval_metric='mean_absolute_error').fit(
        train_subset_ag, presets='medium_quality', ag_args_fit={'num_gpus': 0}, included_model_types=['CAT']
    )

    # Predict on the test subset
    y_pred = predictor.predict(test_subset_ag)
    
    # Assign the predictions to the final_predictions DataFrame
    final_predictions.loc[test_subset.index, 'CI_HOUR'] = y_pred.values

# Merge the final predictions with sample_submission on the index
sample_submission['CI_HOUR'] = final_predictions['CI_HOUR']
sample_submission.to_csv("testing.csv", index=False)


No path specified. Models will be saved in: "AutogluonModels\ag-20231012_031040\"
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231012_031040\"
AutoGluon Version:  0.8.2
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   18.96 GB / 511.09 GB (3.7%)
Train Data Rows:    41214
Train Data Columns: 53
Label Column: CI_HOUR
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (2158.901944, 0.0, 108.52434, 288.64109)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeat

GIW5
GIW5
GIW5
GIW5
GIW5
41214
41214
41214
41214
41214


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 9): ['ARI_CO', 'ARI_PO', 'PORT_SIZE', 'mean_enc_ARI_CO', 'std_enc_ARI_CO', 'mean_enc_ARI_PO', 'std_enc_ARI_PO', 'country_cluster', 'PORT_SIZE_Zone']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Types of features in original data (raw dtype, special dtypes):
		('float', [])  : 28 | ['DIST', 'LENGTH', 'DUBAI', 'BRENT', 'WTI', ...]
		('int', [])    :  9 | ['DEADWEIGHT', 'GT', 'ATA_LT', 'year', 'month', ...]

NGG6
NGG6
NGG6
NGG6
NGG6
27243
27243
27243
27243
27243


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 9): ['ARI_CO', 'ARI_PO', 'PORT_SIZE', 'mean_enc_ARI_CO', 'std_enc_ARI_CO', 'mean_enc_ARI_PO', 'std_enc_ARI_PO', 'country_cluster', 'PORT_SIZE_Zone']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Types of features in original data (raw dtype, special dtypes):
		('float', [])  : 28 | ['DIST', 'LENGTH', 'DUBAI', 'BRENT', 'WTI', ...]
		('int', [])    :  9 | ['DEADWEIGHT', 'GT', 'ATA_LT', 'year', 'month', ...]

...

		('object', []) :  7 | ['SHIP_TYPE_CATEGORY', 'ID', 'BREADTH', 'DEPTH', 'DRAUGHT', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('category', [])  :  6 | ['SHIP_TYPE_CATEGORY', 'ID', 'BREADTH', 'DRAUGHT', 'FLAG', ...]
		('float', [])     : 28 | ['DIST', 'LENGTH', 'DUBAI', 'BRENT', 'WTI', ...]
		('int', [])       :  9 | ['DEADWEIGHT', 'GT', 'ATA_LT', 'year', 'month', ...]
		('int', ['bool']) :  1 | ['DEPTH']
	0.1s = Fit runtime
	44 features in original data used to generate 44 features in processed data.
	Train Data (Processed) Memory Usage: 0.14 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.18s ...
AutoGluon will gauge predictive performance using evaluation metric: 'mean_absolute_error'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.
	To change this, specify the eval_metric parameter of Predictor()
Automatically generating train

LHD1
LHD1
LHD1
LHD1
LHD1
433
433
433
433
433


	Useless Original Features (Count: 10): ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'PORT_SIZE', 'mean_enc_ARI_CO', 'std_enc_ARI_CO', 'mean_enc_ARI_PO', 'std_enc_ARI_PO', 'country_cluster', 'PORT_SIZE_Zone']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Types of features in original data (raw dtype, special dtypes):
		('float', [])  : 28 | ['DIST', 'LENGTH', 'DUBAI', 'BRENT', 'WTI', ...]
		('int', [])    :  9 | ['DEADWEIGHT', 'GT', 'ATA_LT', 'year', 'month', ...]
		('object', []) :  6 | ['ID', 'BREADTH', 'DEPTH', 'DRAUGHT', 'FLAG', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('category', [])  :  4 | ['ID', 'BREADTH', 'FLAG', 'PO_Y_M_D']
		('float', [])     : 28 | ['DIST', 'LENGTH', 'DUBAI', 'BRENT', 'WTI', ...]
		('int', [])       :  9 | ['DEADWEIGHT', 'GT', 'ATA_LT', 'year', 'month', ...]
		('

OOV8
OOV8
OOV8
OOV8
OOV8
416
416
416
416
416


		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Types of features in original data (raw dtype, special dtypes):
		('float', [])  : 28 | ['DIST', 'LENGTH', 'DUBAI', 'BRENT', 'WTI', ...]
		('int', [])    :  9 | ['DEADWEIGHT', 'GT', 'ATA_LT', 'year', 'month', ...]
		('object', []) :  7 | ['SHIP_TYPE_CATEGORY', 'ID', 'BREADTH', 'DEPTH', 'DRAUGHT', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('category', [])  :  4 | ['SHIP_TYPE_CATEGORY', 'ID', 'FLAG', 'PO_Y_M_D']
		('float', [])     : 28 | ['DIST', 'LENGTH', 'DUBAI', 'BRENT', 'WTI', ...]
		('int', [])       :  9 | ['DEADWEIGHT', 'GT', 'ATA_LT', 'year', 'month', ...]
		('int', ['bool']) :  3 | ['BREADTH', 'DEPTH', 'DRAUGHT']
	0.1s = Fit runtime
	44 features in original data used to generate 44 features in processed data.
	Train Data (Processed) Memor

In [37]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset(train)
test_data = TabularDataset(test)


#predictor = TabularPredictor(label='CI_HOUR',  eval_metric='mean_absolute_error').fit(train, presets='medium_quality',  ag_args_fit={'num_gpus': 0}, included_model_types = ['CAT'])
#predictor = TabularPredictor(label='CI_HsOUR',  eval_metric='mean_absolute_error').fit(train, presets='medium_quality',  ag_args_fit={'num_gpus': 0})
predictor = TabularPredictor(label='CI_HOUR',  eval_metric='mean_absolute_error').fit(train, presets='medium_quality',  ag_args_fit={'num_gpus': 0}, included_model_types = ['RF', 'LGBM'])

predictor.leaderboard(train, silent=True)


y_pred = predictor.predict(test_data)
y_pred = pd.DataFrame(y_pred, columns=['CI_HOUR'])
submission['CI_HOUR'] = y_pred['CI_HOUR']
predictor.leaderboard(train, silent=True)

No path specified. Models will be saved in: "AutogluonModels\ag-20231012_073236\"
Presets specified: ['medium_quality']
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231012_073236\"
AutoGluon Version:  0.8.2
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   14.72 GB / 511.09 GB (2.9%)
Train Data Rows:    367441
Train Data Columns: 6
Label Column: CI_HOUR
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (2159.130556, 0.0, 61.87712, 170.57522)
	If 'regression' is not the correct problem_type, please manually specify the

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE,-10.569477,-17.825751,3.979716,0.060516,46.566810,3.979716,0.060516,46.566810,1,True,1
1,WeightedEnsemble_L2,-10.569477,-17.825751,4.009856,0.061610,46.573476,0.030141,0.001094,0.006665,2,True,2


# feature_selection

In [40]:
from itertools import combinations
from autogluon.tabular import TabularDataset, TabularPredictor

# 데이터 로드
train_data = TabularDataset(train)

# 타겟 변수와 모든 피처를 리스트로 변환
target = 'CI_HOUR'
all_features = [col for col in train_data.columns if col != target]

# 모든 피처들에서 하나씩 제외한 조합 생성
all_combinations = list(combinations(all_features, len(all_features) - 1))

best_score = float('inf')  # MAE는 낮을수록 좋으므로 초기값을 무한대로 설정
best_combination = None
iteration = 0
output_path = 'results.txt'

for feature_combination in all_combinations:
    iteration += 1
    subset_data = train_data[[target] + list(feature_combination)]
    
    predictor = TabularPredictor(label=target, eval_metric='mean_absolute_error').fit(subset_data, presets='medium_quality', ag_args_fit={'num_gpus': 0}, included_model_types=['RF', 'GBM', 'XGB'])
    lb = predictor.leaderboard(subset_data, silent=True)
    score = lb['score_val'].max()
    
    # 결과를 txt 파일에 계속 추가
    with open(output_path, 'a') as f:
        f.write("Iteration: " + str(iteration) + "\n")
        f.write("Excluded feature: " + ', '.join(set(all_features) - set(feature_combination)) + "\n")
        f.write("MAE: " + str(score) + "\n")
        f.write("-" * 50 + "\n")  # 구분선
    
    if score < best_score:
        best_score = score
        best_combination = feature_combination

print("Best feature combination:", best_combination)
print("Best MAE:", best_score)


No path specified. Models will be saved in: "AutogluonModels\ag-20231012_073722\"
Presets specified: ['medium_quality']
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231012_073722\"
AutoGluon Version:  0.8.2
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   13.94 GB / 511.09 GB (2.7%)
Train Data Rows:    367441
Train Data Columns: 5
Label Column: CI_HOUR
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (2159.130556, 0.0, 61.87712, 170.57522)
	If 'regression' is not the correct problem_type, please manually specify the

[1000]	valid_set's l1: 77.3625
[2000]	valid_set's l1: 73.131
[3000]	valid_set's l1: 70.167
[4000]	valid_set's l1: 67.8511
[5000]	valid_set's l1: 65.8352
[6000]	valid_set's l1: 64.1543
[7000]	valid_set's l1: 62.7938
[8000]	valid_set's l1: 61.5584
[9000]	valid_set's l1: 60.4681
[10000]	valid_set's l1: 59.4697


	-59.4697	 = Validation score   (-mean_absolute_error)
	53.31s	 = Training   runtime
	0.97s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's l1: 57.0162
[2000]	valid_set's l1: 47.248
[3000]	valid_set's l1: 41.5582
[4000]	valid_set's l1: 37.6642
[5000]	valid_set's l1: 35.1605
[6000]	valid_set's l1: 33.3518
[7000]	valid_set's l1: 31.6402
[8000]	valid_set's l1: 30.4382


KeyboardInterrupt: 

# Pycaret

In [18]:
from pycaret.regression import *
import category_encoders

reg = setup(data=train, target='CI_HOUR', use_gpu=True, session_id=724)
reg.pipeline

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more 

,Description,Value
0,Session id,724
1,Target,CI_HOUR
2,Target type,Regression
3,Original data shape,"(391939, 34)"
4,Transformed data shape,"(391939, 61)"
5,Transformed train set shape,"(274357, 61)"
6,Transformed test set shape,"(117582, 61)"
7,Numeric features,27
8,Categorical features,6
9,Rows with missing values,42.0%


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more 

Pipeline(memory=FastMemory(location=C:\Users\ineeji\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['DIST', 'BREADTH', 'BUILT',
                                             'DEADWEIGHT', 'DEPTH', 'DRAUGHT',
                                             'GT', 'LENGTH', 'U_WIND', 'V_WIND',
                                             'AIR_TEMPERATURE', 'BN', 'ATA_LT',
                                             'PORT_SIZE', 'year', 'month',
                                             'day', 'weekday', 'rounded_hour',
                                             'month_sin', 'month_cos',
                                             'day_sin', 'da...
                 TransformerWrapper(include=['ARI_CO', 'SHIP_TYPE_CATEGORY'],
                                    transformer=OneHotEncoder(cols=['ARI_CO',
                                                                    'SHIP_TYPE_CATEGORY'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('rest_encoding',
                 TransformerWrapper(include=['ARI_PO', 'ID', 'SHIPMANAGER',
                                             'FLAG'],
                                    transformer=TargetEncoder(cols=['ARI_PO',
                                                                    'ID',
                                                                    'SHIPMANAGER',
                                                                    'FLAG'],
                                                              handle_missing='return_nan')))])

In [23]:
best = compare_models(n_select=5, sort='MAE')
best

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,54.9854,29459.0022,171.6006,-0.0167,1.6046,1.5096,8.5900
et,Extra Trees Regressor,55.6215,23147.8583,152.1201,0.2009,1.1782,7.9439,49.7030
lightgbm,Light Gradient Boosting Machine,56.1851,23288.6768,152.5767,0.1961,1.3281,8.2002,3.1570
rf,Random Forest Regressor,56.2211,23626.3472,153.6832,0.1843,1.1180,9.1540,74.7390
catboost,CatBoost Regressor,56.5754,22502.2461,149.9807,0.2232,1.5418,7.7854,7.3630
xgboost,Extreme Gradient Boosting,56.9273,22891.6742,151.2825,0.2096,1.5614,7.0685,3.1840
gbr,Gradient Boosting Regressor,59.6339,24076.1137,155.1372,0.1689,1.5893,8.2068,77.8910
knn,K Neighbors Regressor,63.9900,26721.4156,163.4444,0.0774,1.8694,5.2598,9.2520
dt,Decision Tree Regressor,68.3752,40239.4473,200.5720,-0.3907,1.2516,9.7241,7.4880
lr,Linear Regression,77.8328,26337.2801,162.2622,0.0908,2.5952,7.7996,3.0120


[HuberRegressor(),
 ExtraTreesRegressor(n_jobs=-1, random_state=724),
 LGBMRegressor(device='gpu', n_jobs=-1, random_state=724),
 RandomForestRegressor(n_jobs=-1, random_state=724),

In [22]:
cat_t = tune_model(best, optimize='MAE', n_iter=100) #lgbma
cat_t

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,14:34:39
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Huber Regressor


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


KeyboardInterrupt: 

In [24]:
stack_lr = stack_models(best, optimize='MAE', choose_better=True)
stack_lr

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:01:36
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Stacking Regressor


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [25]:
stack_finalized = finalize_model(best)
stack_finalized

ValueError: Estimator [HuberRegressor(), ExtraTreesRegressor(n_jobs=-1, random_state=724), LGBMRegressor(device='gpu', n_jobs=-1, random_state=724), RandomForestRegressor(n_jobs=-1, random_state=724), <catboost.core.CatBoostRegressor object at 0x0000028D89C95040>] does not have the required fit() method.

In [56]:
submission = pd.read_csv('(f)bestaccuracy_nofit_scaled_sample_submission.csv')
all = pd.concat([test,submission],axis=1)
all['CI_HOUR'][all['DIST'] == 0] = 0
submission['CI_HOUR'] = all['CI_HOUR']
submission['CI_HOUR'][submission['CI_HOUR'] < 0] = 0
submission.to_csv('(f)bestaccuracy_nofit_scaled_sample_submission_2.csv',index=False)
submission

,SAMPLE_ID,CI_HOUR
0,TEST_000000,90.917114
1,TEST_000001,339.572906
2,TEST_000002,0.000000
3,TEST_000003,0.000000
4,TEST_000004,24.792130
...,...,...
244984,TEST_244984,130.493073
244985,TEST_244985,383.237549
244986,TEST_244986,0.000000
244987,TEST_244987,0.000000


In [40]:
submission['CI_HOUR'] = stack_finalized.predict(test)
all = pd.concat([test,submission],axis=1)
all['CI_HOUR'][all['DIST'] == 0] = 0
submission['CI_HOUR'] = all['CI_HOUR']
submission['CI_HOUR'][submission['CI_HOUR'] < 0] = 0
submission.to_csv('pycat.csv',index=False)
submission

,SAMPLE_ID,CI_HOUR
0,TEST_000000,98.878138
1,TEST_000001,206.534507
2,TEST_000002,0.000000
3,TEST_000003,0.000000
4,TEST_000004,67.811210
...,...,...
244984,TEST_244984,29.505385
244985,TEST_244985,205.425372
244986,TEST_244986,0.000000
244987,TEST_244987,0.000000


# jar

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
from sklearn.metrics import mean_squared_error


'''
automl = AutoML(
    algorithms=["CatBoost", "Xgboost", "LightGBM"],
    model_time_limit=30*60,
    start_random_models=10,
    hill_climbing_steps=3,
    top_models_to_improve=3,
    golden_features=True,
    features_selection=False,
    stack_models=True,
    train_ensemble=True,
    explain_level=0,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)
'''
automl = AutoML(mode = "Compete",
                algorithms = ['LightGBM', 'Xgboost', 'CatBoost'], golden_features=True,
                ml_task = "regression", eval_metric=mean_absolute_error, random_state = 42, total_time_limit=100)

automl.fit(train_x, train_y)

predictions = automl.predict(test)
predictions

AutoML directory: AutoML_1
The task is regression with evaluation metric user_defined_metric
AutoML will use algorithms: ['LightGBM', 'Xgboost', 'CatBoost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree user_defined_metric 79.057654 trained in 0.52 seconds
Disable stacking for split validation
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 3 models
There was an error during 2_Default_LightGBM training.
Please check AutoML_1\errors.md for details.
There was an error during 3_Default_Xgboost training.
Please check AutoML_1\errors.md for details.
There was an error

array([63.11427878, 77.93100383, 63.11427878, ..., 59.73600808,
       63.11427878, 63.11427878])

# h2o

In [11]:
!pip install h2o

In [12]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()
# Assuming your data is in a pandas DataFrame called 'df'
h2o_frame = h2o.H2OFrame(train)
target = 'CI_HOUR'
features = h2o_frame.columns
features.remove(target)
# Set up the AutoML object
aml = H2OAutoML(max_runtime_secs=3600, # Adjust as needed
                seed=313,
                stopping_metric="MAE", # Setting MAE as the stopping metric
                sort_metric="MAE")     # Setting MAE as the sorting metric for the leaderboard

# Train the model
aml.train(x=features, y=target, training_frame=train)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...


H2OStartupError: Cannot find Java. Please install the latest JRE from
http://docs.h2o.ai/h2o/latest-stable/h2o-docs/welcome.html#java-requirements

In [ ]:
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))


In [ ]:
preds = aml.predict(test)


In [ ]:
mae = h2o.h2o.mae(preds, test[target])
print(f"Mean Absolute Error on Test Set: {mae}")


In [ ]:
h2o.cluster().shutdown()


# CAT

In [13]:
from xgboost import XGBRegressor
#import xgboost as xgb
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import numpy as np


def objective(trial: Trial, X_train, y_train, X_val, y_val):
    params = {
            'iterations':trial.suggest_int("iterations", 300, 1000),
            'learning_rate' : trial.suggest_uniform('learning_rate',0.1, 1),
            'depth': trial.suggest_int('depth',5, 16),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
            'reg_lambda': trial.suggest_uniform('reg_lambda',30,100),
            'subsample': trial.suggest_uniform('subsample',0.3,1),
            'random_strength': trial.suggest_uniform('random_strength',10,100),
            'od_wait':trial.suggest_int('od_wait', 10, 150),
            'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,20),
            'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 1, 100),
            "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0., 1.0),
            'random_state' : 313,
            'verbose' : 0,
        }
    #'task_type' : 'GPU',
    #"eval_metric":'RMSE',
    cat = CatBoostRegressor(**params)
    cat.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_val,y_val)],cat_features=cat_features,
              verbose=False)
    cat_pred = cat.predict(X_val)
    score = mean_absolute_error(y_val, cat_pred)
    return score


In [14]:
from sklearn.preprocessing import LabelEncoder
import bisect
'''
# Categorical 컬럼 인코딩
categorical_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']
encoders = {}

for feature in tqdm(categorical_features, desc="Encoding features"):
    le = LabelEncoder()
    train[feature] = le.fit_transform(train[feature].astype(str))
    le_classes_set = set(le.classes_)
    test[feature] = test[feature].map(lambda s: '-1' if s not in le_classes_set else s)
    le_classes = le.classes_.tolist()
    bisect.insort_left(le_classes, '-1')
    le.classes_ = np.array(le_classes)
    test[feature] = le.transform(test[feature].astype(str))
    encoders[feature] = le
cat_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']

# 결측치 처리
train_x.fillna(train_x.mean(), inplace=True)
test.fillna(train_x.mean(), inplace=True)
'''
cat_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'country_cluster', 'PORT_SIZE_Zone', 'BREADTH', 'DEPTH', 'DRAUGHT', 'PO_Y_M_D', 'FLAG']
# 수치형 변수만 대상으로 결측치 대체
numeric_cols = train_x.select_dtypes(include=[np.number]).columns

# 훈련 데이터의 평균 계산
mean_values = train_x[numeric_cols].mean()

# 결측치 대체
train_x[numeric_cols] = train_x[numeric_cols].fillna(mean_values)
test[numeric_cols] = test[numeric_cols].fillna(mean_values)

In [15]:
# 모든 카테고리형 변수를 문자열로 변환
categorical_cols = train_x.select_dtypes(include=['object', 'category']).columns
train_x[categorical_cols] = train_x[categorical_cols].astype(str)
test[categorical_cols] = test[categorical_cols].astype(str)


In [16]:
kf = KFold(n_splits=10, shuffle=True, random_state=1008)
fold_results = []
test_predictions = []

# 각 Fold에 대해서 Optuna로 하이퍼파라미터 튜닝
for fold, (train_index, val_index) in enumerate(kf.split(train_x)):
    X_train, X_val = train_x.iloc[train_index, :], train_x.iloc[val_index, :]
    y_train, y_val = train_y.iloc[train_index, :], train_y.iloc[val_index, :]

    print(f"Optimizing hyperparameters for fold {fold+1}...")
    study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=313))
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val.values), n_trials=20)

    best_params = study.best_params
    best_score = study.best_value
    
    print(f"Best RMSE for fold {fold+1}: {best_score}")
    print(f"Best hyperparameters for fold {fold+1}: {best_params}")
    
    fold_results.append({
        'fold': fold+1,
        'best_score': best_score,
        'best_params': best_params
    })
    
    # 최적의 하이퍼파라미터로 테스트 데이터 예측
    model = CatBoostRegressor(**best_params)
    model.fit(X_train, y_train, cat_features=cat_features)
    test_pred = model.predict(test)
    test_predictions.append(test_pred)

# 평균 앙상블
final_prediction = np.mean(test_predictions, axis=0)

# 결과 출력
for result in fold_results:
    print(f"Fold {result['fold']}, Best MAE: {result['best_score']}, Best hyperparameters: {result['best_params']}")

# 최종 예측 저장
final_prediction_df = pd.DataFrame(final_prediction, columns=['MLM'])
submission['CI_HOUR'] = final_prediction

[I 2023-10-13 16:57:11,821] A new study created in memory with name: no-name-4f7cc427-49e0-4f9c-b8ec-3af8b8181bd9
[W 2023-10-13 16:57:11,824] Trial 0 failed with parameters: {'iterations': 416, 'learning_rate': 0.5950939294198955, 'depth': 15, 'min_data_in_leaf': 19, 'reg_lambda': 96.23724709797567, 'subsample': 0.6926005800768529, 'random_strength': 88.26287372368104, 'od_wait': 34, 'leaf_estimation_iterations': 10, 'bagging_temperature': 11.278863324734598, 'colsample_bylevel': 0.43633414956249306} because of the following error: ValueError("'ARI_CO' is not in list").
Traceback (most recent call last):
  File "c:\Users\ineeji\anaconda3\envs\v1\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ineeji\AppData\Local\Temp\ipykernel_4244\2899142785.py", line 12, in <lambda>
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val.values), n_trials=20)
  File "C:\Users\ineeji\AppData\Local\Temp\i

Optimizing hyperparameters for fold 1...


ValueError: 'ARI_CO' is not in list

In [100]:
all = pd.concat([test,submission],axis=1)
all['CI_HOUR'][all['DIST'] == 0] = 0
submission['CI_HOUR'] = all['CI_HOUR']
submission['CI_HOUR'][submission['CI_HOUR'] < 0] = 0
submission.to_csv('CAT_v2_1008.csv',index=False)
submission

,SAMPLE_ID,CI_HOUR
0,TEST_000000,97.257813
1,TEST_000001,336.809458
2,TEST_000002,0.000000
3,TEST_000003,0.000000
4,TEST_000004,68.130348
...,...,...
244984,TEST_244984,156.044942
244985,TEST_244985,417.542500
244986,TEST_244986,0.000000
244987,TEST_244987,0.000000


# LGBM

In [ ]:
from xgboost import XGBRegressor
#import xgboost as xgb
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import lightgbm as lgb

def objective(trial: Trial, X_train, y_train, X_val, y_val):
    param = {
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'mae', 
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }
    model = lgb.LGBMRegressor(**param)
    model.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_val,y_val)],
            early_stopping_rounds=50,
            verbose=100)
    y_pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)
    return mae


In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=3732)
fold_results = []
test_predictions = []

# 각 Fold에 대해서 Optuna로 하이퍼파라미터 튜닝
for fold, (train_index, val_index) in enumerate(kf.split(train_x)):
    X_train, X_val = train_x.iloc[train_index, :], train_x.iloc[val_index, :]
    y_train, y_val = train_y.iloc[train_index, :], train_y.iloc[val_index, :]

    print(f"Optimizing hyperparameters for fold {fold+1}...")
    study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=1234))
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val.values), n_trials=20)

    best_params = study.best_params
    best_score = study.best_value
    
    print(f"Best MAE for fold {fold+1}: {best_score}")
    print(f"Best hyperparameters for fold {fold+1}: {best_params}")
    
    fold_results.append({
        'fold': fold+1,
        'best_score': best_score,
        'best_params': best_params
    })
    
    # 최적의 하이퍼파라미터로 테스트 데이터 예측
    model = lgb.LGBMRegressor(**best_params)
    model.fit(X_train, y_train)
    test_pred = model.predict(test)
    test_predictions.append(test_pred)

# 평균 앙상블
final_prediction = np.mean(test_predictions, axis=0)

# 결과 출력
for result in fold_results:
    print(f"Fold {result['fold']}, Best MAE: {result['best_score']}, Best hyperparameters: {result['best_params']}")

# 최종 예측 저장
final_prediction_df = pd.DataFrame(final_prediction, columns=['CI_HOUR'])
submission['CI_HOUR'] = final_prediction

In [58]:
train

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,nAcid,nBase,SpAbs_A,...,MACCS_key_158,MACCS_key_159,MACCS_key_160,MACCS_key_161,MACCS_key_162,MACCS_key_163,MACCS_key_164,MACCS_key_165,MLM,HLM
0,3.259,400.495,5,2,8,3.259,117.37,0,0,35.689316,...,1,1,1,1,1,1,1,1,26.010,50.680
1,2.169,301.407,2,1,2,2.172,73.47,0,0,26.575899,...,1,0,1,1,1,1,1,1,29.270,50.590
2,1.593,297.358,5,0,3,1.585,62.45,2,1,29.802128,...,1,0,1,1,1,1,0,1,5.586,80.892
3,4.771,494.652,6,0,5,3.475,92.60,0,1,45.884166,...,1,1,1,1,1,1,1,1,5.710,2.000
4,2.335,268.310,3,0,1,2.337,42.43,0,0,26.308663,...,1,1,1,1,1,1,1,1,93.270,99.990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,3.409,396.195,3,1,5,3.409,64.74,0,0,30.902711,...,1,0,1,1,1,0,1,1,1.556,3.079
3467,1.912,359.381,4,1,3,1.844,77.37,0,0,35.887372,...,1,1,1,1,1,1,1,1,35.560,47.630
3468,1.941,261.320,3,1,6,2.124,70.14,0,0,23.546531,...,1,1,1,1,1,1,1,1,56.150,1.790
3469,0.989,284.696,5,1,5,0.989,91.51,0,0,23.936088,...,1,1,0,1,1,1,1,1,0.030,2.770
